In [2]:
import pickle
import math
import numpy as np
import matplotlib.pyplot as plt
from os.path import join
from s2and.data import ANDData
%matplotlib inline

2022-11-04 13:17:55,015 - s2and - WARNING - You haven't set `main_data_dir` in data/path_config.json! Using data/ as default data directory.


In [6]:
from tqdm import tqdm

In [2]:
# load s2and production model -> Clusterer obj
with open("data/production_model.pickle", "rb") as _pkl_file:
    prod_model = pickle.load(_pkl_file)
    clusterer = prod_model['clusterer']

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

In [3]:
def create_model_and_eval(X_train, y_train, X_val, y_val):
    pairwise_model = PairwiseModeler(
        n_iter=25, monotone_constraints=featurization_info.lightgbm_monotone_constraints
    )
    pairwise_model.fit(X_train, y_train, X_val, y_val)
    y_pred = pairwise_model.predict_proba(X_val)
    acc = np.sum((y_pred[:, 1]>0.5).astype(int) == y_val) / len(y_pred)
    return acc

In [3]:
# load dataset in "train" mode

from s2and.model import PairwiseModeler
from s2and.featurizer import FeaturizationInfo, featurize
from s2and.eval import pairwise_eval

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

In [8]:
# Strategy 1: Impute to 0

def strategy1(train, val):
    X_train, y_train, _ = train
    X_val, y_val, _ = val

    X_train = np.nan_to_num(X_train)
    X_val = np.nan_to_num(X_val)

    acc = create_model_and_eval(X_train, y_train, X_val, y_val)
    return acc

In [9]:
# Strategy 2: Impute to -1

def strategy2(train, val):
    X_train, y_train, _ = train
    X_val, y_val, _ = val

    X_train = np.nan_to_num(X_train, nan=-1)
    X_val = np.nan_to_num(X_val, nan=-1)

    acc = create_model_and_eval(X_train, y_train, X_val, y_val)
    return acc

In [10]:
# Strategy 3: Do not impute; skip in split decisions

def strategy3(train, val):
    X_train, y_train, _ = train
    X_val, y_val, _ = val

    acc = create_model_and_eval(X_train, y_train, X_val, y_val)
    return acc

In [11]:
# Strategy 4: Set to large negative value

def strategy4(train, val):
    X_train, y_train, _ = train
    X_val, y_val, _ = val

    X_train = np.nan_to_num(X_train, nan=-10000)
    X_val = np.nan_to_num(X_val, nan=-10000)

    acc = create_model_and_eval(X_train, y_train, X_val, y_val)
    return acc

In [12]:
# Strategy 5: Mean imputation

def strategy5(train, val):
    X_train, y_train, _ = train
    X_val, y_val, _ = val

    col_mean = np.nanmean(X_train, axis=0)
    inds = np.where(np.isnan(X_train))
    X_train[inds] = np.take(col_mean, inds[1])
    inds = np.where(np.isnan(X_val))
    X_val[inds] = np.take(col_mean, inds[1])

    acc = create_model_and_eval(X_train, y_train, X_val, y_val)
    return acc

In [15]:
acc_per_strategy = [[]]*5
datasets = ["pubmed", "qian", "zbmath"]
strategy_fns = [strategy1, strategy2, strategy3, strategy4, strategy5]

for dataset_name in datasets:
    print(f"Evaluating over {dataset_name}:\n")
    parent_dir = f"data/{dataset_name}"
    dataset = ANDData(
        signatures=join(parent_dir, f"{dataset_name}_signatures.json"),
        papers=join(parent_dir, f"{dataset_name}_papers.json"),
        mode="train",
        specter_embeddings=join(parent_dir, f"{dataset_name}_specter.pickle"),
        clusters=join(parent_dir, f"{dataset_name}_clusters.json"),
        block_type="s2",
        train_pairs_size=100000,
        val_pairs_size=10000,
        test_pairs_size=10000,
        name=dataset_name,
        n_jobs=8,
    )

    featurization_info = FeaturizationInfo()
    # the cache will make it faster to train multiple times - it stores the features on disk for you
    train, val, test = featurize(dataset, featurization_info, n_jobs=8, use_cache=True)
    
    for i, strategy_fn in tqdm(enumerate(strategy_fns)):
        acc = strategy_fn(train, val)
        acc_per_strategy[i].append(acc)

2022-11-01 15:51:38,969 - s2and - INFO - loading papers


Evaluating over pubmed:



2022-11-01 15:51:41,146 - s2and - INFO - loaded papers
2022-11-01 15:51:41,147 - s2and - INFO - loading signatures
2022-11-01 15:51:42,588 - s2and - INFO - loaded signatures
2022-11-01 15:51:42,589 - s2and - INFO - loading clusters
2022-11-01 15:51:42,593 - s2and - INFO - loaded clusters, loading specter
2022-11-01 15:51:42,617 - s2and - INFO - loaded specter, loading cluster seeds
2022-11-01 15:51:42,618 - s2and - INFO - loaded cluster seeds
2022-11-01 15:51:42,619 - s2and - INFO - making signature to cluster id
2022-11-01 15:51:42,622 - s2and - INFO - made signature to cluster id
2022-11-01 15:51:42,622 - s2and - INFO - loading name counts
2022-11-01 15:51:58,313 - s2and - INFO - loaded name counts
2022-11-01 15:51:58,721 - s2and - INFO - preprocessing papers
Preprocessing papers 2/2: 100%|████████████████████████████| 134623/134623 [00:47<00:00, 2816.45it/s]
2022-11-01 15:53:14,574 - s2and - INFO - preprocessed papers
2022-11-01 15:53:14,575 - s2and - INFO - preprocessing signatures


 80%|███████████████████████▏     | 20/25 [06:52<01:40, 20.18s/trial, best loss: -0.9998516009462936]
                                                                                                  [LightGBM] [Warning] 

 80%|███████████████████████▏     | 20/25 [06:52<01:40, 20.18s/trial, best loss: -0.9998516009462936]
                                                                                                  Monotone penalty greater than tree depth. Monotone features won't be used.

 88%|█████████████████████████▌   | 22/25 [07:24<00:56, 18.88s/trial, best loss: -0.9998516009462936]
                                                                                                  [LightGBM] [Warning] 

 88%|█████████████████████████▌   | 22/25 [07:24<00:56, 18.88s/trial, best loss: -0.9998516009462936]
                                                                                                  Monotone penalty greater than tree depth. Monotone features won't be used.

1

1it [09:27, 567.05s/it]

...

4it [1:06:30, 848.37s/it]


  0%|                                                         | 0/25 [00:00<?, ?trial/s, best loss=?]

Mean of empty slice



100%|█████████████████████████████| 25/25 [14:42<00:00, 35.30s/trial, best loss: -0.9540756025730451]


5it [1:21:38, 979.65s/it]


In [16]:
accs = np.array(acc_per_strategy)
accs = np.mean(accs, axis=1)
accs

array([0.94778985, 0.94778985, 0.94778985, 0.94778985, 0.94778985])

In [38]:
# acc_per_strategy[0]
accs = []
for i in range(3):
    for k,j in enumerate(range(i*5, i*5 + 5)):
        if i == 0:
            accs.append([])
        accs[k].append(acc_per_strategy[0][j])

In [39]:
accs

[[0.9973252703802767, 0.8792134831460674, 0.9782],
 [0.9973252703802767, 0.8728152309612984, 0.9728],
 [0.9986044888940574, 0.8748439450686641, 0.9776],
 [0.9973252703802767, 0.8728152309612984, 0.9728],
 [0.9981393185254099, 0.8528401997503121, 0.9742]]

In [40]:
np.mean(accs, axis=1)

array([0.95157958, 0.94764683, 0.95034948, 0.94764683, 0.94172651])

In [41]:
np.argmax(np.mean(accs, axis=1))

# Best validation performance observed for Strategy 1: impute 0 for nan

0

In [5]:
datasets = ["pubmed", "qian", "zbmath"]  
# skipping -- "inspire", "medline"
# done -- "aminer", "arnetminer", "kisti"

missing = [0.257931452991453, 0.191869137094056, 0.17408965811965812]
lengths = [300000, 281168, 300000]

for dataset_name in datasets:
    parent_dir = f"data/{dataset_name}"
    dataset = ANDData(
        signatures=join(parent_dir, f"{dataset_name}_signatures.json"),
        papers=join(parent_dir, f"{dataset_name}_papers.json"),
        mode="train",
        specter_embeddings=join(parent_dir, f"{dataset_name}_specter.pickle"),
        clusters=join(parent_dir, f"{dataset_name}_clusters.json"),
        block_type="s2",
        train_pairs_size=100000,
        val_pairs_size=100000,
        test_pairs_size=100000,
        name=dataset_name,
        n_jobs=8,
    )

    featurization_info = FeaturizationInfo()
    # the cache will make it faster to train multiple times - it stores the features on disk for you
    train, val, test = featurize(dataset, featurization_info, n_jobs=8, use_cache=True)
    
    missing_pct = (np.sum(np.isnan(train[0])) + np.sum(np.isnan(val[0])) + np.sum(np.isnan(test[0]))) / \
    (np.prod(train[0].shape) + np.prod(val[0].shape) + np.prod(test[0].shape))
    missing.append(missing_pct)
    
    total_pairs = train[0].shape[0] + val[0].shape[0] + test[0].shape[0]
    lengths.append(total_pairs)
    
    print(missing_pct, total_pairs)
    del dataset, train, val, test

print("missing_pcts:", missing)
print("missing_pct_avg", np.mean(missing))
print("n_pairs:", lengths)
print("n_pairs_mean:", np.mean(lengths))

2022-11-04 17:42:11,917 - s2and - INFO - loading papers
2022-11-04 17:42:17,794 - s2and - INFO - loaded papers
2022-11-04 17:42:17,795 - s2and - INFO - loading signatures
2022-11-04 17:42:20,395 - s2and - INFO - loaded signatures
2022-11-04 17:42:20,395 - s2and - INFO - loading clusters
2022-11-04 17:42:20,399 - s2and - INFO - loaded clusters, loading specter
2022-11-04 17:42:20,528 - s2and - INFO - loaded specter, loading cluster seeds
2022-11-04 17:42:20,528 - s2and - INFO - loaded cluster seeds
2022-11-04 17:42:20,529 - s2and - INFO - making signature to cluster id
2022-11-04 17:42:20,531 - s2and - INFO - made signature to cluster id
2022-11-04 17:42:20,531 - s2and - INFO - loading name counts
2022-11-04 17:42:36,632 - s2and - INFO - loaded name counts
2022-11-04 17:42:37,034 - s2and - INFO - preprocessing papers
Preprocessing papers 2/2: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 134623/1

0.2617637716462652 125525


2022-11-04 17:44:02,216 - s2and - INFO - loaded papers
2022-11-04 17:44:02,217 - s2and - INFO - loading signatures
2022-11-04 17:44:03,861 - s2and - INFO - loaded signatures
2022-11-04 17:44:03,861 - s2and - INFO - loading clusters
2022-11-04 17:44:03,868 - s2and - INFO - loaded clusters, loading specter
2022-11-04 17:44:04,133 - s2and - INFO - loaded specter, loading cluster seeds
2022-11-04 17:44:04,134 - s2and - INFO - loaded cluster seeds
2022-11-04 17:44:04,135 - s2and - INFO - making signature to cluster id
2022-11-04 17:44:04,137 - s2and - INFO - made signature to cluster id
2022-11-04 17:44:04,138 - s2and - INFO - loading name counts
2022-11-04 17:44:20,162 - s2and - INFO - loaded name counts
2022-11-04 17:44:20,380 - s2and - INFO - preprocessing papers
Preprocessing papers 2/2: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 59545/59545 [00:21<00:00, 2764.76it/s]
2022-11-04 17:44:58,704

0.17877380072502025 113652


2022-11-04 17:45:05,401 - s2and - INFO - loaded papers
2022-11-04 17:45:05,402 - s2and - INFO - loading signatures
2022-11-04 17:45:05,559 - s2and - INFO - loaded signatures
2022-11-04 17:45:05,559 - s2and - INFO - loading clusters
2022-11-04 17:45:05,580 - s2and - INFO - loaded clusters, loading specter
2022-11-04 17:45:05,995 - s2and - INFO - loaded specter, loading cluster seeds
2022-11-04 17:45:05,995 - s2and - INFO - loaded cluster seeds
2022-11-04 17:45:05,996 - s2and - INFO - making signature to cluster id
2022-11-04 17:45:06,001 - s2and - INFO - made signature to cluster id
2022-11-04 17:45:06,002 - s2and - INFO - loading name counts
2022-11-04 17:45:22,060 - s2and - INFO - loaded name counts
2022-11-04 17:45:22,159 - s2and - INFO - preprocessing papers
Preprocessing papers 2/2: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 23406/23406 [00:04<00:00, 4909.45it/s]
2022-11-04 17:45:39,154

0.4144615842850388 147724
missing_pcts: [0.257931452991453, 0.191869137094056, 0.17408965811965812, 0.2617637716462652, 0.17877380072502025, 0.4144615842850388]
missing_pct_avg 0.24648156747691521
n_pairs: [300000, 281168, 300000, 125525, 113652, 147724]
n_pairs_mean: 211344.83333333334
